# The Randomized Benchmark Protocol 
------------

- Randomized Benchmarking Protocol (RB) is a technique used in quantum information science to
assess the quality of quantum gates and operations in a quantum computing system. Quantum gates are the
fundamental building blocks of quantum circuits, and their accurate implementation is crucial for the reliable
execution of quantum algorithms. RB provides a standardized way to quantify the error rates of these gates and
to characterize the overall performance of a quantum processor.
- Basic Idea: RB involves applying a sequence of random gate operations (also known as "cliffords") to a quantum
state, followed by an inverse sequence of gates to return the state to its original form. The error accumulation
during this process provides insight into the overall gate quality.
- Randomized Operations (Clifford Gates):
RB employs sequences of random Clifford gates, which are a well-defined set of quantum gates with known
mathematical properties. Clifford gates are chosen because they are easily implementable and form a universal
set, meaning they can be combined to approximate any quantum operation.

In [1]:
!pip install qiskit

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/ibapi-10.30.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install qiskit-aer

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/ibapi-10.30.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector, state_fidelity

In [4]:

# Generate a random quantum circuit
def generate_random_circuit(num_qubits, depth):
    circuit = QuantumCircuit(num_qubits)
    for _ in range(depth):
        for qubit in range(num_qubits):
            circuit.rx(np.random.uniform(0, 2 * np.pi), qubit)
            circuit.ry(np.random.uniform(0, 2 * np.pi), qubit)
            circuit.rz(np.random.uniform(0, 2 * np.pi), qubit)
        for qubit in range(num_qubits - 1):
            circuit.cz(qubit, qubit + 1)
    return circuit

In [5]:
# Perform randomized benchmarking
def randomized_benchmarking(num_qubits, depths, num_sequences, shots):
    backend = Aer.get_backend('aer_simulator')  # AerSimulator backend
    results = []
    for depth in depths:
        success_counts = 0
        for _ in range(num_sequences):
            # Generate a random circuit and its inverse
            circuit = generate_random_circuit(num_qubits, depth)
            inverse_circuit = circuit.inverse()

            # Transpile the circuits for the backend
            transpiled_circuit = transpile(circuit, backend)
            transpiled_inverse_circuit = transpile(inverse_circuit, backend)

            # Apply the circuit and obtain the final statevector
            transpiled_circuit.save_statevector()  # Ensure statevector is saved
            circuit_result = backend.run(transpiled_circuit).result()  # Run the transpiled circuit
            final_statevector = Statevector(circuit_result.get_statevector())

            # Apply the inverse circuit and obtain the final statevector
            transpiled_inverse_circuit.save_statevector()  # Ensure statevector is saved
            inverse_result = backend.run(transpiled_inverse_circuit).result()  # Run the transpiled inverse circuit
            inverse_statevector = Statevector(inverse_result.get_statevector())

            # Calculate the fidelity
            fidelity = state_fidelity(final_statevector, inverse_statevector)
            success_counts += shots * (1 - fidelity)

        success_rate = success_counts / (num_sequences * shots)
        results.append(success_rate)
    return results

# Example usage
num_qubits = 2
depths = [1, 2, 3, 4]
num_sequences = 100
shots = 1024

results = randomized_benchmarking(num_qubits, depths, num_sequences, shots)
print(results)

[0.6163714915531966, 0.7158548447750991, 0.7022418097119681, 0.7116425701146474]
